In [1]:
import requests
import bs4
import re
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [2]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}

In [3]:
url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
response = requests.get(url, headers=HEADERS)
soup = bs4.BeautifulSoup(response.content, 'html.parser')

In [4]:
MAIN_URL = 'https://www.imdb.com'

def imdb(soup):
    imdb_source_url = soup.find('ul', class_='ipc-metadata-list ipc-metadata-list--dividers-between sc-9d2f6de0-0 iMNUXk compact-list-view ipc-metadata-list--base')
    imdb_href = [link['href'] for link in imdb_source_url.select('a.ipc-title-link-wrapper')]
    imdb_href_join = [urljoin(MAIN_URL, imdb_href) for imdb_href in imdb_href]
    return imdb_href_join

imdb_movie = imdb(soup)
imdb_movie

['https://www.imdb.com/title/tt0111161/?ref_=chttp_t_1',
 'https://www.imdb.com/title/tt0068646/?ref_=chttp_t_2',
 'https://www.imdb.com/title/tt0468569/?ref_=chttp_t_3',
 'https://www.imdb.com/title/tt0071562/?ref_=chttp_t_4',
 'https://www.imdb.com/title/tt0050083/?ref_=chttp_t_5',
 'https://www.imdb.com/title/tt0108052/?ref_=chttp_t_6',
 'https://www.imdb.com/title/tt0167260/?ref_=chttp_t_7',
 'https://www.imdb.com/title/tt0110912/?ref_=chttp_t_8',
 'https://www.imdb.com/title/tt0120737/?ref_=chttp_t_9',
 'https://www.imdb.com/title/tt0060196/?ref_=chttp_t_10',
 'https://www.imdb.com/title/tt0109830/?ref_=chttp_t_11',
 'https://www.imdb.com/title/tt0137523/?ref_=chttp_t_12',
 'https://www.imdb.com/title/tt0167261/?ref_=chttp_t_13',
 'https://www.imdb.com/title/tt1375666/?ref_=chttp_t_14',
 'https://www.imdb.com/title/tt0080684/?ref_=chttp_t_15',
 'https://www.imdb.com/title/tt0133093/?ref_=chttp_t_16',
 'https://www.imdb.com/title/tt0099685/?ref_=chttp_t_17',
 'https://www.imdb.com/

In [5]:
movie_250_list = []
NUM = 0

for movie in range(len(imdb_movie)):
    try:
        response = requests.get(imdb_movie[movie], headers=HEADERS)
        response.raise_for_status()
        soup = bs4.BeautifulSoup(response.content, 'html.parser')

        def movie_title(soup):
            extract_movie_title = soup.find('h1', {'data-testid':'hero__pageTitle'}).text.strip()
            return extract_movie_title
        extract_movie_title = movie_title(soup)

        def country_pl(soup):
            country = soup.find('li', {'data-testid': 'title-details-origin'})
            extract_country = country.find('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link').text
            return extract_country
        extract_country = country_pl(soup)

        def runtime(soup):
            get_runtime = soup.find('div', {'data-testid': 'title-techspecs-section'})
            if get_runtime:
                time = get_runtime.find('div', class_='ipc-metadata-list-item__content-container')
                if time:
                    extract_runtime = time.text.strip()
                return extract_runtime
        extract_runtime = runtime(soup)

        def imdb(soup):
            get_imdb = soup.find('div', {'data-testid': 'hero-rating-bar__aggregate-rating__score'})
            if get_imdb:
                get_score = get_imdb.find('span')
                if get_score:
                    extract_get_score = re.search(r'^([\d.]+)', get_score.get_text(strip=True))
                    if extract_get_score:
                        extract_imdb_score = extract_get_score.group(1)
                        return extract_imdb_score
        extract_imdb_score = imdb(soup)

        def meta_score(soup):
            get_meta_score = soup.find('span', class_='sc-b0901df4-0 bcQdDJ metacritic-score-box')
            if get_meta_score:
                extract_meta_score = get_meta_score.text.strip()
                return extract_meta_score
        extract_meta_score = meta_score(soup)
       
        def award(soup):
            oscars_award = soup.find('ul', class_='ipc-metadata-list ipc-metadata-list--dividers-none sc-b45a339a-2 cEeAGk ipc-metadata-list--base')
            if oscars_award:
                extract_award = oscars_award.get_text(strip=True)
            return extract_award
        extract_award = award(soup)

        def genre(soup):
            genres = soup.find_all('a', class_='ipc-chip ipc-chip--on-baseAlt', href=lambda x: x and 'genres=' in x)
            get_genres = [genre.find('span', class_='ipc-chip__text').get_text(strip=True) for genre in genres]
            extract_movie_genres = ', '.join(get_genres)
            return extract_movie_genres
        extract_movie_genres = genre(soup)

        def release(soup):
            release_dates = soup.find_all('a', class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color')
            for date in release_dates:
                extract_release_date = re.search(r'(\d{4})', date.get_text(strip=True))
                if extract_release_date:
                    extract_release_date = extract_release_date.group(1)
                    break
            return extract_release_date
        extract_release_date = release(soup)

        def presentation(soup):
            extract_presentation = soup.find('span', {'data-testid': 'plot-l'}).get_text(strip=True)
            return extract_presentation
        extract_presentation = presentation(soup)

        def review(soup):
            get_review = soup.find('div', {'data-testid':'review-overflow'})
            if get_review:
                extract_review = get_review.text.strip()
            return extract_review
        extract_review = review(soup)

        def video(soup):
            url_video = soup.find('a', {'class':'ipc-title-link-wrapper'})
            if url_video:
                href_value = url_video.get('href')
                extract_video = [urljoin(MAIN_URL, href_value)]
            return extract_video
        extract_video = video(soup)

        NUM += 1
        movie_250_list.append({
            ' ': NUM,
            'Movie': extract_movie_title,
            'Countries of origin': extract_country,
            'Runtime': extract_runtime,
            'IMDb rating': extract_imdb_score,
            'Metascore': extract_meta_score,
            'Award': extract_award,
            'Genre': extract_movie_genres,
            'Release date': extract_release_date,
            'Presentation': extract_presentation,
            'Featured reviews': extract_review,
            'Video': extract_video
        })

    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error occurred: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Request Exception: {err}")
    except Exception as e:
        print(f"An error occurred: {e}")


In [6]:
imdb_movie = pd.DataFrame(movie_250_list, columns = [' ', 'Movie', 'Countries of origin', 'Runtime', 'IMDb rating', 'Metascore', 'Award', 'Genre', 'Release date', 'Presentation', 'Featured reviews', 'Video'])
imdb_movie

,,Movie,Countries of origin,Runtime,IMDb rating,Metascore,Award,Genre,Release date,Presentation,Featured reviews,Video
0,1,Nhà tù Shawshank,United States,2 hours 22 minutes,9.3,82,Nominated for 7 Oscars21 wins & 42 nominations...,Drama,1994,"Over the course of several years, two convicts...",This movie is not your ordinary Hollywood flic...,[https://www.imdb.com/title/tt0111161/videogal...
1,2,Bố Già,United States,2 hours 55 minutes,9.2,100,Won 3 Oscars30 wins & 31 nominations total,"Crime, Drama",1972,"Don Vito Corleone, head of a mafia family, dec...","The acting was simply amazing, what else could...",[https://www.imdb.com/title/tt0068646/videogal...
2,3,Kỵ Sĩ Bóng Đêm,United States,2 hours 32 minutes,9.0,84,Won 2 Oscars162 wins & 164 nominations total,"Action, Crime, Drama",2008,When the menace known as the Joker wreaks havo...,"Dark, yes, complex, ambitious. Christopher Nol...",[https://www.imdb.com/title/tt0468569/videogal...
3,4,Bố Già Phần II,United States,3 hours 22 minutes,9.0,90,Won 6 Oscars17 wins & 21 nominations total,"Crime, Drama",1974,The early life and career of Vito Corleone in ...,The Godfather Part 2 is the finest sequel ever...,[https://www.imdb.com/title/tt0071562/videogal...
4,5,12 Người Đàn Ông Giận Dữ,United States,1 hour 36 minutes,9.0,97,Nominated for 3 Oscars16 wins & 12 nominations...,"Crime, Drama",1957,The jury in a New York City murder trial is fr...,"I watched this film for the first time, when i...",[https://www.imdb.com/title/tt0050083/videogal...
...,...,...,...,...,...,...,...,...,...,...,...,...
245,246,400 Cú Đấm,France,1 hour 39 minutes,8.1,None,Nominated for 1 Oscar8 wins & 5 nominations total,"Crime, Drama",1959,"A young boy, left without attention, delves in...","Every day life, however 'real' and gritty it m...",[https://www.imdb.com/title/tt0053198/videogal...
246,247,Aladdin,United States,1 hour 30 minutes,8.0,86,Won 2 Oscars34 wins & 22 nominations total,"Animation, Adventure, Comedy",1992,A kind-hearted street urchin and a power-hungr...,"I did not care to see this movie, but I was su...",[https://www.imdb.com/title/tt0103639/videogal...
247,248,Khiêu Vũ Với Bầy Sói,United States,3 hours 1 minute,8.0,72,Won 7 Oscars55 wins & 38 nominations total,"Adventure, Drama, Western",1990,"Lieutenant John Dunbar, assigned to a remote w...",It's hard to think of blockbuster films that p...,[https://www.imdb.com/title/tt0099348/videogal...
248,249,Drishyam,India,2 hours 43 minutes,8.2,None,Awards2 wins & 8 nominations,"Crime, Drama, Mystery",2015,Desperate measures are taken by a man who trie...,A great movie and superb story line!!!. A must...,[https://www.imdb.com/title/tt4430212/videogal...


In [7]:
imdb_movie.to_csv('IMDb Top 250 Movies.csv', index=False)